In [31]:
import gzip
import io 
import requests
import csv
import numpy as np

url_list = ['https://data.darts.isas.jaxa.jp/pub/akari/AKARI-FIS_Catalogue_AllSky_BrightSource_1.0/AKARI-FIS_BSC_V1.txt.gz', 
       'https://www.ir.isas.jaxa.jp/AKARI/Archive/Catalogues/FISBSCv2/AKARI-FIS_BSC_V2_main.txt.gz', 
       'https://www.ir.isas.jaxa.jp/AKARI/Archive/Catalogues/FISBSCv2/AKARI-FIS_BSC_V2_supl.txt.gz']
path = "../Data/akari_data.csv"

num_lines = 500_000

def extract(url, max_lines):
    response = requests.get(url, stream=True, auth=('fisbsc','grade=3'))
    response.raise_for_status()

    line_count = 0
    
    with gzip.GzipFile(fileobj=io.BytesIO(response.content)) as gz:
        with io.TextIOWrapper(gz, encoding='utf-8') as reader:
            for line in reader:
                if not line.strip():
                    continue
                fields = line.strip().split()
                
                line_count += 1
                if line_count > max_lines:
                    break
                try:
                    flux1 = float(fields[6])
                    flux2 = float(fields[8])
                    flux3 = float(fields[10])
                    flux4 = float(fields[12])

                    mg65 = -2.5*np.log10(flux1)
                    mg90 = -2.5*np.log10(flux2)
                    mg140 = -2.5*np.log10(flux3)
                    mg160 = -2.5*np.log10(flux4)
                except (ValueError, IndexError, ZeroDivisionError, TypeError):
                    continue
                
                try:
                    ra = float(fields[2])
                    dec = float(fields[3])
                    qf65 = int(fields[14])
                    qf90 = int(fields[15])
                    qf140 = int(fields[16])
                    qf160 = int(fields[17])
                    #bf65 = fields[18]
                    #bf90 = fields[19]
                    #bf140 = fields[20]
                    #bf160 = fields[21]
                except (IndexError, ValueError):
                    continue
                quality_cond = all([
                    qf65 >= 2,
                    qf90 >= 2,
                    qf140 >= 2,
                    qf160 >= 2
                ])
                color_cond = (mg65 > mg90 > mg140 > mg160)
                condt = quality_cond and color_cond
                if condt:
                    print('dih')
                    yield ra, dec, bf65, bf90, bf140, bf160

def process_files(url_list):
    for url in url_list:
        print('ye')
        yield from extract(url, num_lines)

with open(path, 'w', newline='') as akari_csv:
    writer = csv.writer(akari_csv)
    writer.writerow(['ra','dec','bf65', 'bf90', 'bf140', 'bf160'])
    for row in process_files(url_list):
        writer.writerow(row)

ye


KeyboardInterrupt: 